# 上下文管理

In [ ]:
应用场景
1.增强功能
在代码执行的前后增强其功能，类似装饰器的功能。
2.资源管理
打开了资源需要关闭，如文件对象、网络连接、数据库连接。
3.权限验证
在执行代码之前，在__enter__中处理。

In [2]:
# 如果业务逻辑简单可以contextlib.contextmanager装饰器，如果业务复杂，则用类的方法__enter__和__exit__

import datetime
import time
import contextlib

@contextlib.contextmanager
def add(x, y):  # 为生成器函数增加了上下文管理
    start = datetime.datetime.now()
    try:
        yield x + y  # yield的值只有一个，作为__enter__方法的返回值
    finally:
        delta = (datetime.datetime.now() - start).total_seconds()
        print(delta)
        
with add(4, 5) as f:
    # raise Exception()
    time.sleep(2)
    print(f)

9
2.000114


# 反射

In [ ]:
reflection，指的是运行时获取类型定义信息。

In [ ]:
三个内建函数：
getattr(object, name[, default]) => 属性不存在，使用default返回，没有default，抛出AttributeError。
getattr(object, name, value) => 存在则覆盖，不存在，则新增
hasattr(object, name) => name必须为字符串

In [ ]:
四个魔术方法：
__getattr__ => 当通过搜索实例、实例的类及祖先类查不到属性，就会调用此方法
__setattr__ => 通过访问实例属性，进行增加、修改，即self.x = x，都要调用此方法
            => 可以拦截对实例属性的增加、修改操作，也可以自己操作把属性添加到实例的__dict__
__delattr__ => 当通过实例来删除属性时调用此方法
            => 可以用来阻止通过实例删除属性的操作，但依然可以通过类来删除。
__getattribute__ => 实例所有的属性调用都从这个方法开始
                 => 为了避免在该方法中无限递归，它的实现应调用基类的同名方法以访问需要的任何属性，
                    如， object.__getattribute__(self, name)。

In [ ]:
属性查找顺序：
实例调用__getattribute__() --> instance.__dict --> instance.__class__.__dict__ --> 继承的祖先类（直到object）的__dict__
--> 找不到 --> 调用__getattr__() 

实例的所有属性访问，第一个都会调用__getattribute__，它阻止了属性的查找，它的return值作为属性查找的结果。
如果抛出AttributeError（必须是AttributeError异常，raise AttributeError()），则会直接调用__getattr__，因为表示属性没有找到。

In [ ]:
这种动态增加属性的方式和装饰器修饰一个类、Mixin方式的差异？
这种方式是运行时改变类或者实例，但另两种都是定义时就决定了，因此反射能力具有更大的灵活性。

# Descriptors/描述器

In [ ]:
三个魔术方法：
object.__get__(self, instance, owner)
object.__set__(self, instance, value)
object.__delete__(self, instance)

In [ ]:
一个类实现了__get__、__set__、__delete__三个方法中的任何一个，就是描述器。

如果仅实现了__get__，就是非数据性描述器 non-data descriptor
如果实现了__set__或__delete__至少一个，就是数据性描述器 data descriptor
如果一个类的类属性设置为描述器，那么它被称为owner属主。

In [ ]:
属性查找顺序：
实例的__dict__优先于非数据描述器，数据描述器优先于实例的__dict__
原因：
数据描述器会把实例额的属性从实例的__dict__中去除掉，造成了数据描述器优先访问的假象。

In [18]:
class A:
    def __init__(self, x):
        self.x = x
        print(2, "A init")
        
    def __set__(self, instance, value):
        print(3, self, instance, value)
        
#     def __delete__(self, instance):  # 依然是数据描述器
#         print(3, self, instance)
        
    def __get__(self, instance, owner):
        print(4, self, instance, owner)
        
class B:
    x = A(7)  # 类的实例，必须是一个实例
    
    def __init__(self):
        print(1, "B init")
        print("*" * 30)
        self.x = "b.x"  # 如果存在__delete__，不存在__set__，会报错AttributeError: __set__
        print(self.__dict__)  # 非数据描述器时，输出为{'x': 'b.x'}；数据描述器时，输出为{}
        print("*" * 30)
        
print(5, B.x)
b = B()
print(6, b.x)

2 A init
4 <__main__.A object at 0x0000000005BBA5F8> None <class '__main__.B'>
5 None
1 B init
******************************
3 <__main__.A object at 0x0000000005BBA5F8> <__main__.B object at 0x0000000005BBA748> b.x
{}
******************************
4 <__main__.A object at 0x0000000005BBA5F8> <__main__.B object at 0x0000000005BBA748> <class '__main__.B'>
6 None


In [ ]:
staticmethod()和classmethod()都是非数据描述器。实例可以重新定义和覆盖方法，允许单个实例获取与其他实例不同的行为。
property()实现是数据描述器。不允许实例不能覆盖属性的行为。

In [21]:
from functools import partial

class StaticMethod:
    def __init__(self, fn):
        self.fn = fn
        
    def __get__(self, instance, owner):
        print("Static Method")
        return self.fn
    
class ClassMethod:
    def __init__(self, fn):
        self.fn = fn
        
    def __get__(self, instance, cls):
        print("Class Method")
        # return self.fn(cls)
        return partial(self.fn, cls)
    
class A:
    @StaticMethod
    def s_mtd():
        print("static method")
      
    @ClassMethod
    def c_mtd(cls):
        print(f"{cls} class method")
        
a = A()
a.s_mtd()
a.c_mtd()

Static Method
static method
Class Method
<class '__main__.A'> class method
